Create a new lakehouse called it TPCH100 or something like that, it is a one off manual operation, then add it to the current notebook

In [1]:
!pip install -q duckdb --pre --upgrade

StatementMeta(, ba58d74d-628c-42a8-a139-3b4c514a9b3f, 3, Finished, Available)

In [2]:
sf =100

StatementMeta(, ba58d74d-628c-42a8-a139-3b4c514a9b3f, 4, Finished, Available)

In [3]:
%%time
import duckdb
import pathlib
for x in range(0, sf) :
  con=duckdb.connect()
  con.sql('PRAGMA disable_progress_bar;SET preserve_insertion_order=false')
  con.sql(f"CALL dbgen(sf={sf} , children ={sf}, step = {x})") 
  for tbl in ['nation','region','customer','supplier','lineitem','orders','partsupp','part'] :
     pathlib.Path(f'/lakehouse/default/Files/{sf}/{tbl}').mkdir(parents=True, exist_ok=True) 
     con.sql(f"COPY (SELECT * FROM {tbl}) TO '/lakehouse/default/Files/{sf}/{tbl}/{x:03d}.parquet' ")
  con.close()

StatementMeta(, ba58d74d-628c-42a8-a139-3b4c514a9b3f, 5, Finished, Available)

In [4]:
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
from pyspark.sql.types import *
def loadFullDataFromSource(table_name):
    df = spark.read.parquet(f'Files/{sf}/' + table_name + '/*.parquet')
    df.write.mode("overwrite").format("delta").save("Tables/" + table_name)
full_tables = [
    'customer',
    'lineitem',
    'nation',
    'orders' ,
    'region',
    'partsupp',
    'supplier' ,
    'part'
    ]

for table in full_tables:
    loadFullDataFromSource(table)

StatementMeta(, ba58d74d-628c-42a8-a139-3b4c514a9b3f, 6, Finished, Available)